## Frangi filter

This notebook loads pre-computed CT (`_0000.nrrd`) and Frangi vesselness (`_0001.nrrd`) maps.

In [22]:
import nrrd
import numpy as np
from skimage.filters import frangi
from tqdm import tqdm
import napari

# ---- Configuration ----
volume_path = '../../sten_0002_0000.nrrd'
label_path  = '../../sten_0002.nrrd'

# ---- Load and normalize ----
print(f"Loading volume: {volume_path}")
data, header = nrrd.read(volume_path)
data = data.astype(np.float32)
data = (data - data.min())/(data.max()-data.min())
label, _ = nrrd.read(label_path)

Loading volume: ../../sten_0002_0000.nrrd


In [ ]:
import nrrd
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.restoration import denoise_tv_chambolle
from skimage.morphology import opening, ball, white_tophat

ct, hdr = nrrd.read('../../sten_0002_0000.nrrd')
ct = ct.astype(np.float32)
ct_norm = (ct - ct.min())/(ct.max() - ct.min())
base = 'sten_0002_0000'

# ——— 1) Define your preprocessing functions ———
def morph_open_bg(vol):
    bg = opening(vol, footprint=ball(25))
    return vol - bg

def highpass(vol):
    bg = gaussian_filter(vol, sigma=20)
    hp = vol - bg
    return (hp - hp.min())/(hp.max()-hp.min())

def tv_tophat(vol):
    tv = denoise_tv_chambolle(vol, weight=0.01)
    th = white_tophat(tv, footprint=ball(5))
    return (th - th.min())/(th.max()-th.min())

preprocs = {
    'morph_open_bg': morph_open_bg,
    'highpass':      highpass,
    'tv_tophat':     tv_tophat
}

# ——— 2) Apply and save ———
for name, fn in preprocs.items():
    print(f"Processing & saving '{name}'…")
    vol_pp = fn(ct_norm).astype(np.float32)
    out_filename = f"{base}_{name}.nrrd"
    nrrd.write(out_filename, vol_pp, hdr)
    print(f"  → Wrote {out_filename}")

Processing & saving 'morph_open_bg'…
  → Wrote sten_0002_0000_morph_open_bg.nrrd
Processing & saving 'highpass'…
  → Wrote sten_0002_0000_highpass.nrrd
Processing & saving 'tv_tophat'…
  → Wrote sten_0002_0000_tv_tophat.nrrd


## Tophat

In [19]:
import nrrd
import numpy as np
from skimage.restoration import denoise_tv_chambolle
from skimage.morphology import white_tophat, ball

# 1) Load & normalize
ct_path     = '../../sten_0002_0000.nrrd'
output_path = 'sten_0002_0000_tvtophat.nrrd'

ct, hdr = nrrd.read(ct_path)
ct = ct.astype(np.float32)
ct_norm = (ct - ct.min())/(ct.max() - ct.min())

# 2) TV + white-tophat
def tv_tophat(vol, tv_weight=0.01, se_radius=5):
    tv = denoise_tv_chambolle(vol, weight=tv_weight)
    se = ball(se_radius)
    wt = white_tophat(tv, footprint=se)
    wt = wt - wt.min()
    wt = wt/(wt.max()+1e-8)
    return wt

pp = tv_tophat(ct_norm, tv_weight=0.01, se_radius=5)

# 3) Save as NRRD
nrrd.write(output_path, pp.astype(np.float32), hdr)
print("Saved preprocessed volume to", output_path)


Saved preprocessed volume to sten_0002_0000_tvtophat.nrrd


In [ ]:
import nrrd
import numpy as np
from scipy.ndimage import median_filter, gaussian_filter
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, opening, ball

# 1) load your tophat volume
pp, hdr = nrrd.read('sten_0002_0000_tvtophat.nrrd')

# 2) tiny 3×3×3 median to kill single‐voxel noise
pp_med = median_filter(pp, size=3)

# 3) gentle Gaussian smoothing (sigma = 1 voxel)
pp_smooth = gaussian_filter(pp_med, sigma=1)

# 4) Save both the smoothed “attention” map and the refined mask
nrrd.write('sten_0002_tvtophat_smooth.nrrd', pp_smooth.astype(np.float32), hdr)


## Frangi

In [16]:
# —— Configuration ——
input_path  = 'sten_0002_0000_tv_tophat.nrrd'        # your CTCA volume

# —— 1) Load CT volume ——
vol, header = nrrd.read(input_path)
vol = vol.astype(np.float32)

v = frangi(
        vol,
        sigmas=(1.4, 11.5, 1.4),
        alpha=1,
        beta=1,
        gamma=None,
        black_ridges=False,
        mode='reflect'
    )

# 7) Save & visualize
nrrd.write('sten_0002_0001_frangi.nrrd', v, header)